# Hierachy generation using PyARXaaS

In [1]:
from pyaaas import ARXaaS
from pyaaas.privacy_models import KAnonymity, LDiversityDistinct
from pyaaas import AttributeType
from pyaaas import Dataset
from pyaaas.models.hierarchy import IntervalHierarchyBuilder, RedactionHierarchyBuilder, OrderHierarchyBuilder
import pandas as pd

#### Create connection to ARXaaS

In [8]:
arxaas = ARXaaS("http://localhost:8080")

#### Fetch data 

In [2]:
data_df = pd.read_csv("../data/data2.csv", sep=";")

In [3]:
data_df

,zipcode,age,salary,disease
0,47677,29,3,gastric ulcer
1,47602,22,4,gastritis
2,47678,27,5,stomach cancer
3,47905,43,6,gastritis
4,47909,52,11,flu
5,47906,47,8,bronchitis
6,47605,30,7,bronchitis
7,47673,36,9,pneumonia
8,47607,32,10,stomach cancer


### Create Redaction based hierarchy

#### 1. Extract column to create hierarchy from

In [5]:
column = data_df["zipcode"].tolist()
column

[47677, 47602, 47678, 47905, 47909, 47906, 47605, 47673, 47607]

#### 2. Create hierarchy builder to use

In [6]:
redaction_based = RedactionHierarchyBuilder()

#### 3. Call the ARXaaS service to create the hierarchy

In [9]:
redaction_hierarchy = arxaas.hierarchy(redaction_based, column)

In [10]:
redaction_hierarchy

[['47677', '4767*', '476**', '47***', '4****', '*****'],
 ['47602', '4760*', '476**', '47***', '4****', '*****'],
 ['47678', '4767*', '476**', '47***', '4****', '*****'],
 ['47905', '4790*', '479**', '47***', '4****', '*****'],
 ['47909', '4790*', '479**', '47***', '4****', '*****'],
 ['47906', '4790*', '479**', '47***', '4****', '*****'],
 ['47605', '4760*', '476**', '47***', '4****', '*****'],
 ['47673', '4767*', '476**', '47***', '4****', '*****'],
 ['47607', '4760*', '476**', '47***', '4****', '*****']]

### Create interval based hierarchy

#### 1. Extract column to create hierarchy from

In [11]:
column = data_df["age"].tolist()
column

[29, 22, 27, 43, 52, 47, 30, 36, 32]

#### 2. Create hierarchy builder to use

In [34]:
interval_based = IntervalHierarchyBuilder()
interval_based.add_interval(18,30, "young-adult")
interval_based.add_interval(30,60, "adult")

#### 3. Call the ARXaaS service to create the hierarchy

In [35]:
interval_hierarchy = arxaas.hierarchy(interval_based, column)

In [36]:
interval_hierarchy

[['29', 'young-adult', '*'],
 ['22', 'young-adult', '*'],
 ['27', 'young-adult', '*'],
 ['43', 'adult', '*'],
 ['52', 'adult', '*'],
 ['47', 'adult', '*'],
 ['30', 'adult', '*'],
 ['36', 'adult', '*'],
 ['32', 'adult', '*']]

### Create Order based hierarchy

#### 1. Extract column to create hierarchy from


In [37]:
column = data_df["disease"].tolist()
column.sw

['gastric ulcer',
 'gastritis',
 'stomach cancer',
 'gastritis',
 'flu',
 'bronchitis',
 'bronchitis',
 'pneumonia',
 'stomach cancer']

#### 2. Strip to uniques and order column values

In [47]:
column = set(column)
column = list(column)
column[1], column[3] = column[3], column[1]
column

['stomach cancer',
 'gastric ulcer',
 'gastritis',
 'pneumonia',
 'bronchitis',
 'flu']

#### 2. Create hierarchy builder to use

In [48]:
order_based = OrderHierarchyBuilder()
order_based.level(0).add_group(3, "stomach-related")
order_based.level(0).add_group(3, "lung-related")

#### 3. Call the ARXaaS service to create the hierarchy*

In [49]:
order_hierarchy = arxaas.hierarchy(order_based, column)

In [50]:
order_hierarchy

[['stomach cancer', 'stomach-related', '*'],
 ['gastric ulcer', 'stomach-related', '*'],
 ['gastritis', 'stomach-related', '*'],
 ['pneumonia', 'lung-related', '*'],
 ['bronchitis', 'lung-related', '*'],
 ['flu', 'lung-related', '*']]

### Example anonymization

In [72]:
dataset = Dataset.from_pandas(data_df)

In [73]:
dataset.set_attribute_type(AttributeType.IDENTIFYING, "salary")

In [74]:
dataset.describe()

data:
  headers:
    ['zipcode', 'age', 'salary', 'disease']
rows:
    [47677, 29, 3, 'gastric ulcer']
    [47602, 22, 4, 'gastritis']
    [47678, 27, 5, 'stomach cancer']
    [47905, 43, 6, 'gastritis']
    [47909, 52, 11, 'flu']
    ...
attributes:
  field_name=zipcode, type=QUASIIDENTIFYING, hierarchy=None
  field_name=age, type=QUASIIDENTIFYING, hierarchy=None
  field_name=salary, type=IDENTIFYING, hierarchy=None
  field_name=disease, type=QUASIIDENTIFYING, hierarchy=None



In [75]:
dataset.set_hierarchy("age", interval_hierarchy)

In [76]:
dataset.set_hierarchy("zipcode", redaction_hierarchy)

In [77]:
dataset.set_hierarchy("disease", order_hierarchy)

In [78]:
anon_result = arxaas.anonymize(dataset=dataset, privacy_models=[KAnonymity(2)])

In [79]:
anon_result.dataset.to_dataframe()

,zipcode,age,salary,disease
0,47***,young-adult,*,stomach-related
1,47***,young-adult,*,stomach-related
2,47***,young-adult,*,stomach-related
3,47***,adult,*,stomach-related
4,47***,adult,*,lung-related
5,47***,adult,*,lung-related
6,47***,adult,*,lung-related
7,47***,adult,*,lung-related
8,47***,adult,*,stomach-related
